### Homological Product Code Construction
* Can view quantum CSS codes as chain complexes
$$C_2 \rightarrow C_1 \rightarrow C_0$$
* Can take tensor products of chain complexes (CSS codes) as per the construction given by Hastings et. al.
* There are two 'theories' when it comes to defining CSS codes in terms of homology theory
    1. 'Single sector' theory - Use a single vector space to define chain complex (Will try this first)
        * Parity-check matrix for X-checks is the boundary operator $\partial$
        * Parity-cehck matrix 
    2. 'Multiple sector' theory - Use 3 vector spaces to define chain complex
* $\text{\underline{1st Test: Utilizing Self-Orthogonal Codes}}$
    * Use 2 $[[7,1,3]]$ codes

In [2]:
from sage.all import *
import numpy as np

In [3]:
def parity_check_to_complex(pc_mat_X: np.array, pc_mat_Z: np.array) -> sage.homology.chain_complex.ChainComplex:
    """ 
    Take X and Z parity-check matrices and produce a boundary operator for CSS code

    Parameters:
    -----------
    pc_mat_X - X parity-check matrix
    pc_mat_Z - Z parity-check matrix

    Returns:
    --------
    Chain complex (C,d)
    """
    num_qubits = len(pc_mat_X[0])
    vec_space = ZZ^(num_qubits)
    x_pc_dict = {}
    z_pc_dict = {}
    for x_check in pc_mat_x:
        x_pc_dict[x_check] = vector(ZZ, x_check)

    for z_check in pc_mat_z:
        z_pc_dict[z_check] = vector(ZZ, z_check)
        
    C_X = vec_space.span([x_pc_dict.values()])
    C_Z = vec_space.span([z_pc_dict.values()])

In [ ]:
# Let us begin by defining our chain complex (single sector theory) for Steane Code
x_pc_mat = np.array([[0,0,0,1,1,1,1], [0,1,1,0,0,1,1,], [1,0,1,0,1,0,1]])
z_pc_mat = np.array([[0,0,0,1,1,1,1], [0,1,1,0,0,1,1,], [1,0,1,0,1,0,1]])

n = 7 # Number of physical qubits in CSS code

# Define vector space
C = VectorSpace(GF(2), n)


# data for chain complex

# base_ring (F_2)

